In [1]:
from google.colab import files
uploaded = files.upload()

Saving ptbxl_database.csv to ptbxl_database.csv


In [2]:
from google.colab import files
uploaded = files.upload()

Saving scp_statements.csv to scp_statements.csv


In [20]:
#A6
# Install required libraries (only if needed)
!pip install -q pandas scikit-learn

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer # Import SimpleImputer

# Step 1: Load the datasets
ptbxl_df = pd.read_csv("/content/ptbxl_database.csv")
scp_df = pd.read_csv("/content/scp_statements.csv")

# Step 2: Extract MI and NORM diagnostic codes
scp_df = scp_df[scp_df['diagnostic_class'].notnull()]
mi_codes = scp_df[scp_df['diagnostic_class'] == 'MI']['Unnamed: 0'].tolist()
norm_codes = scp_df[scp_df['diagnostic_class'] == 'NORM']['Unnamed: 0'].tolist()

# Step 3: Filter only MI and NORM cases from ptbxl_df
def label_filter(code_dict):
    codes = eval(code_dict).keys()
    mi = any(code in mi_codes for code in codes)
    norm = any(code in norm_codes for code in codes)
    return (mi and not norm) or (norm and not mi)  # Exclude overlap

filtered_df = ptbxl_df[ptbxl_df['scp_codes'].apply(label_filter)].copy() # Added .copy() to avoid SettingWithCopyWarning

# Assign class labels (either MI or NORM)
def assign_label(code_dict):
    codes = eval(code_dict).keys()
    if any(code in mi_codes for code in codes):
        return 'MI'
    elif any(code in norm_codes for code in codes):
        return 'NORM'
    else:
        return 'OTHER'

filtered_df['label'] = filtered_df['scp_codes'].apply(assign_label)

# Step 4: Feature selection (age and sex)
# Impute missing values in 'age' and 'sex'
imputer = SimpleImputer(strategy='mean')
filtered_df[['age', 'sex']] = imputer.fit_transform(filtered_df[['age', 'sex']])


# Convert sex to numeric (should be done after imputation if imputation on original data)
# Since we are imputing on the filtered_df, sex should already be numeric or imputed as mean
# based on previous step. Let's ensure it's numeric after imputation.
filtered_df['sex'] = filtered_df['sex'].astype(int)


# Prepare features (X) and labels (y)
X = filtered_df[['age', 'sex']].values
y = filtered_df['label'].values

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 6: Train kNN classifier with k = 3
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

# Step 7: Make predictions and evaluate
y_pred = neigh.predict(X_test)

# Output evaluation metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))
# Calculate and print the accuracy on test set
accuracy = neigh.score(X_test, y_test)
print("Test Accuracy of kNN (k=3):", accuracy)



Classification Report:
              precision    recall  f1-score   support

          MI       0.53      0.57      0.55      1629
        NORM       0.75      0.71      0.73      2866

    accuracy                           0.66      4495
   macro avg       0.64      0.64      0.64      4495
weighted avg       0.67      0.66      0.66      4495

Accuracy Score: 0.6593993325917686
Test Accuracy of kNN (k=3): 0.6593993325917686


In [3]:
from google.colab import files
uploaded = files.upload()

Saving 01000_lr.dat to 01000_lr.dat
Saving 01000_lr.hea to 01000_lr.hea
Saving 01001_lr.dat to 01001_lr.dat
